In [52]:
import os
import glob
import pandas as pd

In [42]:
# To do still:
#    Calculate CTCF
#        CTCF = Integrated Density – (Area of selected ROI X Mean fluorescence of background readings)
#    Plot, analyze CTCF values

In [122]:
path =r'/Users/michaelpiacentino/Drive/git/data/smpd3/20180616_CRISPR_TCFLef_sections/csvs/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe
for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    # Split values in ROI label and Image name columns
    df['Fluor'], df['ROI'] = zip(*df['Label'].map(lambda x: x.split(':')))
    df['Target'], df['Method'], df['Embryo'], df['Section'] = zip(*df['Image'].map(lambda x: x.split('_')))
    
    # Replace background ROI names
        # NOTE: I have updated the Fiji macro ('FluorIntensity_2Channel.ijm') to name all background ROIs as 'background',
        # so this step will be unnecessary with freshly collected data
    df.replace(to_replace=['back1a', 'back1b', 'back1c', 'back2a', 'back2b', 'back2c'],
                            value=['background', 'background', 'background', 'background', 'background', 'background'], 
                            inplace=True)
    list_.append(df)

full_df = pd.concat(list_)
full_df.head()

,,Label,Area,Mean,IntDen,RawIntDen,Image,Fluor,ROI,Target,Method,Embryo,Section
0,1,TCFLef:back1a,34.937,13.963,487.825,9467.0,SMPD3_CRISPR_Emb1_sec1,TCFLef,background,SMPD3,CRISPR,Emb1,sec1
1,2,TCFLef:back1b,37.565,13.726,515.599,10006.0,SMPD3_CRISPR_Emb1_sec1,TCFLef,background,SMPD3,CRISPR,Emb1,sec1
2,3,TCFLef:back1c,42.202,13.664,576.661,11191.0,SMPD3_CRISPR_Emb1_sec1,TCFLef,background,SMPD3,CRISPR,Emb1,sec1
3,4,TCFLef:Cntl,7227.200,130.882,945909.299,18356834.0,SMPD3_CRISPR_Emb1_sec1,TCFLef,Cntl,SMPD3,CRISPR,Emb1,sec1
4,5,TCFLef:Expt,7117.289,126.978,903740.336,17538480.0,SMPD3_CRISPR_Emb1_sec1,TCFLef,Expt,SMPD3,CRISPR,Emb1,sec1


In [126]:
# Mean background values and group
grouped_means = (full_df.groupby(['Embryo', 'Fluor', 'ROI'])['Area', 'Mean', 'IntDen', 'RawIntDen']).mean()
grouped_means.head(12)

Area        Mean        IntDen     RawIntDen
Embryo Fluor  ROI                                                            
Emb1   TCFLef Cntl        5925.491667  179.376000  9.568730e+05  1.856960e+07
              Expt        5973.465000  126.157333  7.257828e+05  1.408494e+07
              background    45.946667   13.857778  6.359537e+02  1.234167e+04
       pCIG   Cntl        5925.491667  166.542667  9.642506e+05  1.871278e+07
              Expt        5973.465000  192.381667  1.125004e+06  2.183244e+07
              background    45.946667   26.874556  1.234852e+03  2.396422e+04
Emb2   TCFLef Cntl        4127.026333  675.873333  2.716187e+06  5.271182e+07
              Expt        3327.623000  409.178333  1.378554e+06  2.675297e+07
              background    53.475556   14.190667  7.568464e+02  1.468778e+04
       pCIG   Cntl        4127.026333  336.768667  1.400675e+06  2.718227e+07
              Expt        3327.623000  265.937333  9.008427e+05  1.748225e+07
              background    53.475556   26.248111  1.409410e+03  2.735178e+04